In [30]:
import json
import glob

import numpy as np
import pandas as pd

from qiskit import transpile
from qiskit.providers.fake_provider import FakeLima
from qiskit.primitives import Estimator
from qiskit.circuit.random import random_circuit

import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.functional import dropout

from torch_geometric.nn import GCNConv, global_mean_pool, Linear, ChebConv, SAGEConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from tqdm.notebook import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns

from blackwater.data.loaders.exp_val import CircuitGraphExpValMitigationDataset
from blackwater.data.generators.exp_val_andrej import exp_value_generator
from utils import generate_random_pauli_sum_op
from estimator import ngem
from scripts.data_setup import load_data

In [31]:
lima = FakeLima()

In [32]:
train_circuits, train_observables, train_ideal_exp_vals, train_noisy_exp_vals, test_circuits, test_observables, test_ideal_exp_vals, test_noisy_exp_vals = load_data('data/circuits/data_small_1')

In [33]:
new_train_circuits = []
for i in range(len(train_circuits)):
    if train_circuits[i].num_qubits == 3:
        new_train_circuits.append(train_circuits[i])

new_test_circuits = []
for i in range(len(test_circuits)):
    if test_circuits[i].num_qubits == 3:
        new_test_circuits.append(test_circuits[i])


In [34]:
train_circuits = new_train_circuits
test_circuits = new_test_circuits

In [35]:
N_FILES = 1
N_ENTRIES_PER_FILE = len(train_circuits)

for i in tqdm_notebook(range(N_FILES)):
    generator = exp_value_generator(
        backend=lima,
        circuits=train_circuits,
        observables=train_observables,
        ideal_exp_vals=train_ideal_exp_vals,
        noisy_exp_vals=train_noisy_exp_vals
    )
    with tqdm_notebook(total=N_ENTRIES_PER_FILE) as t:
        entries = []
        for entry in generator:
            entries.append(entry.to_dict())
            t.update(1)
            
        with open(f"data/ngem/train.json", "w") as f:
            json.dump(entries, f)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/231 [00:00<?, ?it/s]

In [36]:
N_FILES = 1
N_ENTRIES_PER_FILE = len(test_circuits)

for i in tqdm_notebook(range(N_FILES)):
    generator = exp_value_generator(
        backend=lima,
        circuits=test_circuits,
        observables=test_observables,
        ideal_exp_vals=test_ideal_exp_vals,
        noisy_exp_vals=test_noisy_exp_vals
    )
    with tqdm_notebook(total=N_ENTRIES_PER_FILE) as t:
        entries = []
        for entry in generator:
            entries.append(entry.to_dict())
            t.update(1)
            
        with open(f"data/ngem/test.json", "w") as f:
            json.dump(entries, f)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/43 [00:00<?, ?it/s]

In [37]:
train_loader = DataLoader(
    CircuitGraphExpValMitigationDataset(
        "data/ngem/train.json",
        transforms=None,
    ),
    batch_size=len(train_circuits),
    shuffle=True
)

val_loader = DataLoader(
    CircuitGraphExpValMitigationDataset(
        "data/ngem/test.json",
        transforms=None,
    ),
    batch_size=len(test_circuits),
    shuffle=False
)

In [38]:
class ExpValCircuitGraphModel(torch.nn.Module):
    def __init__(
        self,
        n_qubits: int,
        num_node_features: int,
        hidden_channels: int
    ):
        super().__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, 1)
        
        self.cheb_conv1 = ChebConv(num_node_features, hidden_channels, K=3)
        self.cheb_conv2 = ChebConv(hidden_channels, 1, K=2)

        self.sage_conv1 = SAGEConv(num_node_features, hidden_channels)
        self.sage_conv2 = SAGEConv(hidden_channels, 1)
        
        self.obs_seq = torch.nn.Sequential(
            Linear(n_qubits * 4 + 1, hidden_channels),
            torch.nn.Dropout(0.2),
            Linear(hidden_channels, 1)
        )
        
        self.body_seq = torch.nn.Sequential(
            Linear(6, hidden_channels),
            Linear(hidden_channels, 1)
        )

    def forward(self, 
                exp_value, observable, 
                circuit_depth, nodes, 
                edge_index, batch):
        # GCN
        graph = self.conv1(nodes, edge_index).relu()
        graph = dropout(graph, training=self.training, p=0.1)
        graph = self.conv2(graph, edge_index).relu()
        graph = dropout(graph, training=self.training, p=0.1)
        graph = self.conv3(graph, edge_index)
        graph = global_mean_pool(graph, batch)
        
        # Cheb
        cheb = self.cheb_conv1(nodes, edge_index).relu()
        cheb = dropout(cheb, training=self.training, p=0.2)
        cheb = self.cheb_conv2(cheb, edge_index)
        cheb = global_mean_pool(cheb, batch)
        
        # Sage
        sage = self.sage_conv1(nodes, edge_index).relu()
        sage = dropout(sage, training=self.training, p=0.2)
        sage = self.sage_conv2(sage, edge_index)
        sage = global_mean_pool(sage, batch)
        
        obs = self.obs_seq(observable)
        obs = torch.mean(obs, dim=1)
        
        merge = torch.cat((
            graph, 
            cheb,
            sage,
            obs,
            circuit_depth,
            exp_value
        ), dim=1)

        return self.body_seq(merge)

In [39]:
model = ExpValCircuitGraphModel(
    n_qubits=5,
    num_node_features=22,
    hidden_channels=32
)
optimizer = Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()
scheduler = ReduceLROnPlateau(optimizer,
                              'min',
                              factor=0.5,
                              patience=20,
                              verbose=True,
                              min_lr=0.000001)


In [40]:
len(train_loader.dataset)

231

In [41]:
min_valid_loss = np.inf

train_losses = []
val_losses = []

N_EPOCHS = 10

progress = tqdm_notebook(range(N_EPOCHS), desc='Model training', leave=True)
for epoch in progress:
    train_loss = 0.0
    model.train()
    for data in train_loader:
        optimizer.zero_grad()

        out = model(
            data.noisy_0,
            data.observable,
            data.circuit_depth,
            data.x,
            data.edge_index,
            data.batch
        )
        loss = criterion(out, data.y)

        train_loss += loss.item()

        loss.backward()
        optimizer.step()

    valid_loss = 0.0
    model.eval()
    for data in val_loader:
        out = model(data.noisy_0, data.observable, data.circuit_depth, data.x, data.edge_index, data.batch)
        loss = criterion(out, data.y)

        valid_loss += loss.item()

    #scheduler.step(valid_loss)

    if epoch >= 1:
        train_losses.append(train_loss / len(train_loader))
        val_losses.append(valid_loss / len(val_loader))

        progress.set_description(f"{round(train_losses[-1], 5)}, {round(val_losses[-1], 5)}")
        progress.refresh()

Model training:   0%|          | 0/10 [00:00<?, ?it/s]

RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 9 but got size 13 for tensor number 1 in the list.